#Implementing Neural Network From Scratch

First we import the required libraries.

In [55]:
import numpy as np
import matplotlib as plt

We now define our Activation Function. Activation Function is used to add non-linearity in our model and normalise the output of our model. In this case, we choose the Sigmoid Function. This function returns values between 0 and 1.

In [56]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

We now define the derivative of the derivative of the Sigmoid Function. This will be used later on in Back Propogation.

In [57]:
def sigmoid_deriv(x):
  return (np.exp(-x)/(1 + np.exp(-x))**2)

Now, to define our neural network we create a class NeuralNetwork. Our neural network takes 3 inputs, passes it through 2 hidden layers of 5 and 4 neurons and then these neurons fire into an output layer of 3 nodes. 

Each neuron corresponds to a particular feature in our dataset. We then assign different weights to the features marking their importance. We then meausre our error by comparing the output with desired output. Now we keep tweaking the weights for a number of iterations until we minimise our error and start getting the desired result.

In Forward Propogation, input to one layer is multiplied by corresponding weights and is passes on to next layer. This proccess is carried out starting from our input layer to the last hidden layer. Then the result is passed onto the output layer. 

In [164]:
class NeuralNetwork(object):
  def __init__(self,Inputs=3,HiddenLayers=[5,4],Outputs=2):
    self.Inputs=Inputs
    self.HiddenLayers=HiddenLayers
    self.Outputs=Outputs

    layers=[self.Inputs]+self.HiddenLayers+[self.Outputs]
# We implement random weights to train the model. Weights are multiplied by inputs at each layer. Weights are a measure of importance of a neuron in getting the desired output from our model. 
    weights = []
    for i in range(len(layers)-1):
      w=np.random.rand(layers[i], layers[i+1])
      weights.append(w)
    self.weights=weights

# We create an empty array for activations. Activation are the activation function applied on result of multiplication of inputs and weights
    activations =[]
    for i in range(len(layers)):
      a=np.zeros(layers[i])
      activations.append(a)
    self.activations=activations

# We create an empty array for derivatives to calculate the gradient
    derivatives=[]
    for i in range(len(layers)-1):
      a=np.zeros((layers[i],layers[i+1]))
      derivatives.append(a)
    self.derivatives=derivatives

# We create a function forward to iterate over layers, apply weight and sigmoid function,get activations and get output after passing through all the hidden layers
  def forward(self,Inputs):
    activation=Inputs
    self.activations[0]=Inputs
    for i, w in enumerate(self.weights):
      unactivatedInputs=np.dot(activation,w)
      activation=sigmoid(unactivatedInputs)
      self.activations[i+1]=activation
    # print(self.activations)
    return activation
# We created a loss function which is a measure of how much the output given by the model deviates from the desired output. 
  def loss(self,Output,RightOutput):
    return (RightOutput-Output)**2
# We created a function to calculate the derivative of loss function.
  def loss_deriv(self,Output,RightOutput):
    return (2*(RightOutput-Output))
# Gradient Descent is an algorithm through which we increase efficiency of training of a neural network on a dataset. In this algorithm, aim is to minimize the "loss function".To do this, we calculate the gradient and move in the direction
# of negative gradient of the loss function in each iteration to get to the minimum. The weights are adjusted accordingly at each iteration.
# So for each iteration we have,
# New weights= weights+(learning rate x Gradient)
  def GradientDescent(self,LearningRate):
    deriv_lr=[]
    for d in self.derivatives:
      for i in range(len(d)):
        d[i]=LearningRate*d[i]
      deriv_lr.append(d)
    New_weights=self.weights+deriv_lr
    self.weights=New_weights
    return New_weights
# We create Back function for back propogation. Through back propogation we calculate gradient of the loss function i.e dL/dW.
# Now we use chain rule of derivative,
# dL/d(W for current layer)=dL/d(activationsfor current layer) x d(activations for current layer)/d(unactivated inputs for current layer) x d(unactivated inputs for current layer)/d(W for previous layer)
  def Back(self,loss_deriv):
    for i in reversed(range(len(self.derivatives))):
        Sigmoid_Deriv=sigmoid_deriv(self.activations[i+1])
        delta=loss_deriv*Sigmoid_Deriv
        delta_reshaped=delta.reshape(delta.shape[0],-1).T
        activation=self.activations[i]
        activation=activation.reshape(activation.shape[0],-1)

        self.derivatives[i]=np.dot(activation,delta_reshaped)

        loss_deriv=np.dot(delta,self.weights[i].T)
        
    return loss_deriv
# We create the Train function giving instructions on how to train the model
  def Train(self,Inputs,RightOutput,epochs,LearningRate):
    losses=[]
    for i in range(epochs):
      Output=self.forward(Inputs[i])

      Loss=self.loss(Output,RightOutput[i])
      losses.append(Loss)
      Loss_Deriv=self.loss_deriv(Output,RightOutput[i])

      self.Back(Loss_Deriv)

      self.GradientDescent(LearningRate)

      print("epoch:{}    loss:{}".format(i+1,Loss))
    return losses

We now create a random dataset and train our model on it

In [165]:
if __name__ =="__main__":
  NN=NeuralNetwork()
  # inputs=[]
  # Inputs=np.random.rand(NN.Inputs)
  # Inputs=Inputs.reshape()
  
  # Creating a random dataset and its correct outputs to train our model 
  dataset=np.random.rand(NN.Inputs,1000) #for i in range(0,NN.Inputs) for j in range(0,1000)])
  dataset=dataset.reshape([1000,3])
  rightOutput=np.random.choice([0,1],size=1000)
# Calling the Train() function to train our dataset
  NN.Train(dataset,rightOutput,100,0.001)

epoch:1    loss:[0.75595361 0.62072168]


ValueError: ignored